In [ ]:
### Simple Predication
import pandas as pd
import numpy as np
from prophet import Prophet

df = pd.DataFrame()
df['ds'] = pd.to_datetime(input_x, unit='ms')
df['y'] = np.array(input_y)

model = Prophet()
model.fit(df)

future = pd.DataFrame()
future['ds'] = pd.to_datetime(output_x, unit='ms')

forecast = model.predict(future)
output_y = forecast['yhat'].tolist()
print(f"result: {output_y}")
return output_y

# Random Forest models

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("https://fuzzy-goldfish-r4rr46q4jp4jf5gv6-5000.app.github.dev/")
model_name = "CO_Predict"
model_version = 1
model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)
future = pd.DataFrame()
future['ds'] = pd.to_datetime(output_x, unit='ms')
future['DAY'] = future['ds'].dt.day
future['MONTH'] = future['ds'].dt.month
future['DAY_OF_WEEK'] = future['ds'].dt.dayofweek

feature_cols = ["DAY", "MONTH", "DAY_OF_WEEK"]
X_input = future[feature_cols]

# Reshape input if necessary
if len(X_input.shape) == 1:
    X_input = X_input.values.reshape(1, -1)
elif X_input.shape[0] == 1:
    X_input = X_input.values.reshape(1, -1)
elif X_input.shape[1] == 1:
    X_input = X_input.values.reshape(-1, 1)

# Make predictions
predictions = model.predict(X_input)

# Convert predictions to list
if isinstance(predictions, pd.DataFrame):
    output_y = predictions['yhat'].tolist()
else:
    output_y = predictions.tolist()

return output_y

ModuleNotFoundError: No module named 'prophet'

# Time Series 

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("https://fuzzy-goldfish-r4rr46q4jp4jf5gv6-5000.app.github.dev/")
model_name = "Time_Series_CO_Predict"
model_version = 1  
model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)
input_y = input_y[-len(output_x):]
df = pd.DataFrame({
    'ds': pd.to_datetime(output_x, unit='ms'),
    'CO': input_y
})
df['hour'] = df['ds'].dt.hour
df['minute'] = df['ds'].dt.minute
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)
for lag in range(1, 4):
    df[f'CO_lag_{lag}'] = df['CO'].shift(lag)
df['CO_roll_mean_3'] = df['CO'].rolling(window=3).mean()
df['CO_roll_std_3'] = df['CO'].rolling(window=3).std()

feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]
df_clean = df.dropna()
X = df_clean[feature_cols]
predictions = model.predict(X)
if isinstance(predictions, pd.DataFrame):
    output_y = predictions['yhat'].tolist()
else:
    output_y = predictions.tolist()
num_dropped_rows = len(df) - len(df_clean)
if num_dropped_rows > 0:
    output_y = [None] * num_dropped_rows + output_y
return output_y